In [2]:
import requests
import re
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import logging

data = pd.DataFrame({
    'URL': [],
    'tel': [],
    'mail': []
})


In [3]:
URL = 'https://c-ns.ru'

PHONE_PATTERNS = [
    r'\+\d{11}',
    r'(\d{4}) \d{3}-\d{3}',
    r'\+\d-\d{3}-\d{3}-\d{4}',
    r'\+\d \d{3} \d{3} \d{2} \d{2}'
    
]



In [4]:
def extract_mail(line):
    match = re.search(r'[\w.+-]+@[\w-]+\.[\w.-]+', line)
    return match.group(0)

def get_phone_numbers(line):
    global PHONE_PATTERNS
    for pattern in PHONE_PATTERNS:
        if len(re.findall(pattern, line)) > 0:
            return np.unique(re.findall(pattern, line))
        

def get_mails(text):
    if '@' in text:
        try:
            return extract_mail(text)
        except:
            pass
        

If information on the page

In [5]:
urls = pd.read_csv('ready_domains.csv', header=None)
urls = urls[urls.columns[0]]
urls[:5]

0                  https://1470umto.ru
1                    https://23city.ru
2                 https://abc-sklad.ru
3                 https://absrealty.ru
4    https://aeroflotovec-2.landorg.ru
Name: 0, dtype: object

In [6]:

def parse_one_url(url):
    req = requests.get(url)
    soup = bs(req.text, 'html.parser')
    texts = soup.find_all(text=True)
    texts = u" ".join(t.strip() for t in texts)
    l = [URL, get_phone_numbers(texts), get_mails(texts)];
    
    return l

In [7]:
parse_one_url(URL)

['https://c-ns.ru', array(['+7 812 610 00 55'], dtype='<U16'), 'info@c-ns.ru']

In [15]:
from multiprocessing import Pool

# This thing is pretty fast
with Pool(processes=32) as mp_pool:
    result = mp_pool.map(parse_one_url, urls[:10])
    
result

[['https://c-ns.ru', None, 'info@1470umto.ru'],
 ['https://c-ns.ru', None, 'info@23city.ru'],
 ['https://c-ns.ru', array(['+79995550055'], dtype='<U12'), None],
 ['https://c-ns.ru',
  array(['+74950237526', '+74951042537', '+74951252640', '+74951564273',
         '+74951810529', '+74954320392', '+74957455533'], dtype='<U12'),
  'u002F96b8b7066d2f4fc49867fc7b8ab981ee@sentry.idacloud.ru'],
 ['https://c-ns.ru', array(['+79625884609'], dtype='<U12'), 'snp.a-2@mail.ru'],
 ['https://c-ns.ru', None, None],
 ['https://c-ns.ru', None, 'akademicheskiy@kortros.ru'],
 ['https://c-ns.ru', None, None],
 ['https://c-ns.ru',
  array(['+74951724469'], dtype='<U12'),
  'info@alcontower.ru'],
 ['https://c-ns.ru', None, None]]

In [21]:
df = pd.DataFrame(result, columns =['url', 'tel', 'mail'], dtype = float)
df


/tmp/ipykernel_11762/1435031242.py:1: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df = pd.DataFrame(result, columns =['url', 'tel', 'mail'], dtype = float)


,url,tel,mail
0,https://c-ns.ru,None,info@1470umto.ru
1,https://c-ns.ru,None,info@23city.ru
2,https://c-ns.ru,[+79995550055],None
3,https://c-ns.ru,"[+74950237526, +74951042537, +74951252640, +74...",u002F96b8b7066d2f4fc49867fc7b8ab981ee@sentry.i...
4,https://c-ns.ru,[+79625884609],snp.a-2@mail.ru
5,https://c-ns.ru,None,None
6,https://c-ns.ru,None,akademicheskiy@kortros.ru
7,https://c-ns.ru,None,None
8,https://c-ns.ru,[+74951724469],info@alcontower.ru
9,https://c-ns.ru,None,None
